<a href="https://colab.research.google.com/github/erictong0/FrederickLab2023/blob/main/Linear%20Regression/Pixel%20Things/230628_Pixelated_Linear_regression_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# irrelevant

In [ ]:
#imports
!pip install bio --upgrade
!pip install scipy

In [ ]:
#imports for timing things
!pip install line_profiler
%load_ext line_profiler

In [ ]:
import sys
import glob
import math
import argparse
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as pltticker
import matplotlib.cm as cm
from matplotlib.patches import Rectangle
import csv
from matplotlib import colors
import re
import pandas as pd
from numpy import savetxt
from sklearn.linear_model import LinearRegression
from statistics import mean
import statsmodels.api as sm
from Bio.Data.IUPACData import protein_letters_1to3 as one2three
import scipy.optimize as spo
from pandas.core.internals.construction import ma
from sklearn.linear_model import Lasso

In [ ]:


#works to load information in variables before the program

fwhm = 2  # Full width at half maximum
# Calculate the standard deviation from the FWHM
c = fwhm / (2 * np.sqrt(2 * np.log(2)))

def get_row_floats(data, row):
  return list(map(lambda x: float(x[row]), data))

xArray = []
for i in range(68):
  b = i / 4 + 54
  xArray.append(np.linspace(b - 3 * c, b + 3 * c, 100))

oData = []
data = []
with open("T_10K.out", mode="r") as infile: #open data file and read contents
  reader = csv.reader(infile, delimiter="\n")
  for row in reader:
    if len(row) == 1:
      data.append(re.split('\t', row[0])[2:])
    else:
      data.append(row)

tempData = []
for i in range(6):
  tempData.append(np.array(get_row_floats(data[1:], i)))

oData.append(tempData)

data = []
tempData = []
count = 0
with open("spotlight 2022.csv", mode="r") as infile: #open data file and read contents
  chunk_size = 1000
  while True:
      chunk = infile.readlines(chunk_size)
      if not chunk:
        break
      for line in chunk:
        reader = csv.reader(infile)
        for row in reader:
          count += 1
          if len(row) == 1:
            tempData.append(re.split(r',', row[0]))
          else:
            tempData.append(row)

data.append(get_row_floats(tempData,0)[1:])
data.append(get_row_floats(tempData,1)[1:])

oData.append(data)

DATAINPUT = oData

In [ ]:
from pickle import FALSE
xScale = 10
yScale = 10

def main(inData, aa, rangeChoices):
  aa1 = aa

  phi = inData[0][0]
  psi = inData[0][1]
  ca = inData[0][2]
  cb = inData[0][3]
  c = inData[0][4]
  n = inData[0][5]



  ca_exp = inData[1][0] #def x as Ca shift
  intensity = inData[1][1]


  atom_key={
            "ca":0,
            "cb":1,
            "co":2,
            "n":3,
          }

  nonCys_nonGly_aas={"A", "D", "E", "F", "H", "I", "K", "L", "M",
                    "N", "P", "Q", "R", "S", "T", "V", "Y", "W"}

  nonCys_aas={"A", "D", "E", "F", "G", "H", "I", "K", "L", "M",
              "N", "P", "Q", "R", "S", "T", "V", "Y", "W"}

  na=np.nan

  aa_spec_ranges={ #ranges of interest for chemical shifts
                  #aa1 Ca      Cb      C         N
                  # Ca & Cb adjusted to 17ppm to match Fig5
                 'A':[[11,28],[203,234],[170,184],[112,133]],
                  'R':[[47,64],[22,39],[169,183],[112,133]],
                  'N':[[44,61],[30,47],[168,182],[106,127]],
                  'D':[[45,62],[33,50],[169,183],[109,130]],
  #               'C':[[50,67],[22,39],[168,182],[105,126]],
                  'Q':[[47,64],[22,39],[169,183],[108,129]],
                  'E':[[49,66],[22,39],[169,183],[109,130]],
                  'G':[[36,53],[203,234],[167,181],[ 98,119]],
                  'H':[[49,66],[23,40],[168,182],[107,128]],
                  'I':[[54,71],[31,48],[168,182],[109,130]],
                  'L':[[45,62],[33,50],[170,184],[110,131]],
                  'K':[[49,66],[26,43],[169,183],[109,130]],
                  'M':[[47,64],[26,43],[169,183],[108,129]],
                  'F':[[50,67],[31,48],[168,182],[107,128]],
                  'P':[[57,74],[25,42],[170,184],[ na, na]],
                  'S':[[50,67],[105,139],[55,72],[167,181],[104,125]],
                  'T':[[54,70],[104,138],[60,77],[168,182],[102,123]],
                  'W':[[48,65],[21,38],[169,183],[109,130]],
                  'Y':[[50,67],[30,47],[168,182],[108,129]],
                  'V':[[54,71],[25,42],[169,183],[108,129]],
                  }

  ####Changed this part####
  rchoice = rangeChoices

  ### extract plot subregions ###
  # "r" stands for ramachandran
  # regions: beta-strand, polyproline helix ii
  #          beta-turn I, alpha-helix,
  #          left handed helix (all phi > 0)
  # !!! top and left corner are always >= or <=
  #     unless they reach the bottom/right axis
  #     (i.e. betaB, ppiiB, left)

  #filter beta regions

  def makeMask(a, b, c, d):
    return np.logical_and(np.logical_and(phi >= a, phi <= b), np.logical_and(psi >= c, psi <= d))

  masks = {}
  datas = {}
  cs_ca = {}
  cs_cb = {}
  cs_co = {}

  for i in range(int(180/xScale)):
    for j in range(int(360/yScale)):
      name = f"{chr(i+65)}{(j+1):02d}"
      mask = makeMask(i*xScale-180, (i+1)*xScale-180, 180-(j+1)*yScale, 180-j*yScale)
      #print((i*10-180, i*10-170, 170-j*10, 180-j*10))
      cs_ca[name] = ca[mask]
      cs_cb[name] = cb[mask]
      cs_co[name] = c[mask]

  name = "rturni"
  mask = makeMask(0, 180, -180, 180)
  cs_ca[name] = ca[mask]
  cs_cb[name] = cb[mask]
  cs_co[name] = c[mask]
  #cs_ca = list(cs_ca) ???

  cs_all = {'ca':cs_ca,
            'cb':cs_cb,
            'c' :cs_co,
            }

  bin_width=0.25
  mtick_spacing=5
  #fig, axs = plt.subplots(1, 1, sharey=False, tight_layout=True,figsize=(10,10)) #35826579

  x_lims = np.array(aa_spec_ranges[aa1][0])
  y_lims = np.array(aa_spec_ranges[aa1][1])
  x_edges = np.arange(x_lims[0],x_lims[1]+bin_width,bin_width) #create an array from one limit to the next using bin widths
  y_edges=np.arange(y_lims[0],y_lims[1]+bin_width,bin_width)
  hw=bin_width/2
  center_bins_x=np.arange(x_lims[0]+hw,x_lims[1]+hw,bin_width)
  center_bins_y=np.arange(y_lims[0]+hw,y_lims[1]+hw,bin_width)
  inputs = []
  h_lists = []
  h_list = []
  c = fwhm / (2 * np.sqrt(2 * np.log(2)))
  for key in cs_ca:
    H, bins = np.histogram(cs_ca[key],bins=(x_edges),density=False)
    for j in range(len(H)):
      a = H[j]  # Maximum value
      b = bins[j]  # Peak position

      x = xArray[round(b*4-216)]

      # Calculate the y values for the curve using the Gaussian function
      y = a * np.exp(-(x - b)**2 / (2 * c**2))
      histogram, bins = np.histogram(x, bins=x_edges, density=False, weights = y)
      h_list.append(histogram)
    h_lists = sum(h_list)
    inputs.append(h_lists)
    #h_lists = []
  H_exp, bins = np.histogram(ca_exp,bins=(x_edges),density=True, weights = intensity)
  #plt.hist(ca_exp, bins = (x_edges), weights = intensity)
  target = (H_exp)


  #print(f"{inputs}\n{np.shape(inputs)}")

  tempArr = []
  for element in inputs:
    isSame = False
    for element2 in tempArr:
      if np.all(element == element2):
        isSame = True
    if isSame:
      pass
    else:
      tempArr.append(element)
  inputs = tempArr

  #print(f"{inputs}\n{np.shape(inputs)}")


  data = {}
  for i in range(len(inputs)):
    data[f'x{i+1}'] = list(inputs[i])
  data['y'] = list(target)

  """hists1 = (inputs[0])
  hists2 = (inputs[1])
  hists3 = (inputs[2])
  hists4 = (inputs[3])
  hists5 = (inputs[4])
  hists6 = (inputs[5])
  hists7 = (inputs[6])
  #print(np.shape(inputs),len(inputs[0]),len(target))
  data = {'x1': list(hists1), 'x2': list(hists2), 'x3': list(hists3), 'x4':list(hists4), 'x5': list(hists5), 'x6':list(hists6), 'x7':list(hists7), 'y': list(target)}"""
  df = pd.DataFrame(data)
  df.to_csv("test.csv")
  #print(df)
  x = df.drop('y', axis=1)
  #x = sm.add_constant(x)
  y = df['y']
  #print(df)

  #lin = Lasso(alpha=0.0001,precompute=True,max_iter=1000,
              #positive=True, random_state=9999, selection='random')
  #results = lin.fit(x,y)

  #a[1]*x[1] + a[2]*x[2] + ... + a[i]*x[i] = y

  spo.least_squares()

  model = sm.OLS(y,x)
  results = model.fit()


  cs_ca_list = [cs_ca[element] for element in cs_ca.keys()]

  #np.savetxt("pd dataframe nano", df.values, fmt='%d')
  return (results, cs_ca_list, H_exp)


def func3P(dict):
  r, s, e = main(DATAINPUT, "T", dict)
  #print(r.summary())
  return (r.summary(), s, e)

resultString, cs_ca_list, H_exp = func3P({})

In [ ]:
coefficientDict = {}
#Fancy regex things to split the total result string into multiple arrays for each line, then further splits each one into the various columns
tempArr = [re.sub(" +", " ", element).split(" ") for element in str(resultString).split("\n")[14:int(180*360/xScale/yScale) + 15]]
#print(tempArr)
r = re.sub(" +", " ", str(resultString).split("\n")[2]).split(" ")[5]
print(tempArr)

In [ ]:
print(resultString)

In [ ]:
tempArr = tempArr[0:236]

In [ ]:
print([element[1] for element in tempArr])

In [ ]:
weightList = [element[1] for element in tempArr]
histList = []
#print(np.shape(weightList), np.shape(cs_ca_list))

#plt.hist(cs_ca_list, bins = np.arange(54, 71, .25))
for i in range(len(cs_ca_list)):
  '''print(cs_ca_list[i])
  print([weightList[i]] * len(cs_ca_list[i]))'''
  if cs_ca_list[i] != []:
    hist, bins1 = np.histogram(cs_ca_list[i], bins = np.arange( 54, 70, .25), weights = [float(weightList[i])] * len(cs_ca_list[i]))
    histList.append(hist)
print(np.shape(bins1), np.shape(histList))
plt.hist(bins1[1:], bins = np.arange(54, 70, .25), weights = sum(histList))
print(np.shape(H_exp))
plt.hist(bins1, bins = np.arange(54,70,.25), weights = H_exp)
print(np.shape(cs_ca_list))
print(np.shape(weightList))

#plt.hist(cs_ca_list, bins = (np.arange(54,71, .25)), weights = weightList)


In [ ]:
#makes a matplotlib rectangle object based on the coordinates of each corner.
def makeRect(s, c):
  return Rectangle((s[0], s[2]), s[1]-s[0], s[3]-s[2], color = c)

#takes a dictionary and plots all of the various rectangles
def visualizeRectangles(d, cs, r2):
  norm = colors.Normalize(-2.7e+07, 2.7e+07)


  fig, ax = plt.subplots()
  for key in d.keys():
    ax.add_patch(makeRect(d[key], cs[key]))

  bound = 180
  ax.set(xlim = (-1 * bound, bound), ylim = (-1*bound, bound))
  plt.xlabel("φ/phi (deg)")
  plt.ylabel("Ψ/psi (deg)")
  plt.title(f"Threonine Data: R²={round(r2, 3)}")
  fig.set_size_inches(6, 6)
  fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cm.RdBu))
  plt.show()

In [ ]:
otherVals = [element[1] for element in tempArr]
print(otherVals)
print(min(otherVals, key=float))
print(max(otherVals, key = float))
tempDict = {}
tempColor = {}
for i in range(100):
  tempDict[i] = (i-180, i-179, -100, 100)
  tempColor[i] = cm.RdBu(i/100)
visualizeRectangles(tempDict, tempColor, -1)

In [ ]:
norm = colors.Normalize(-2.7e+07, 2.7e+07)
visDict = {}
colorArray = {}
valDict = {}
for i in range(int(180/xScale)):
  for j in range(int(360/yScale)):
    visDict[f"{chr(i+65)}{(j+1):02d}"] = (i*xScale-180, (i+1)*xScale-180, 180-(j+1)*yScale, 180-j*yScale)
    val = float(tempArr[int((i)*360/yScale + (j))][1])
    #print(f"{np.log(abs(val))} / {val}")
    valDict[f"{chr(i+65)}{(j+1):02d}"] = val
    colorArray[f"{chr(i+65)}{(j+1):02d}"] = cm.RdBu(norm(val))

visDict["rturni"] = (0, 180, -180, 180)
#print(visDict)
colorArray["rturni"] = cm.RdBu(float(tempArr[-1][1]))
#print(valDict)
visualizeRectangles(visDict, colorArray, float(r))

print(colorArray)


#otherVal = min(max(np.log(val*-1)*-1/4,0),1)
#otherVal = min(max(np.log(float(tempArr[-1][1]))*-1/4,0),1)

In [ ]:
xList = []
yList = []
for i in range(18):
  for j in range(36):
    xList.append(j)
    yList.append(float(tempArr[(i)*36 + (j)][1]))
    if abs(float(tempArr[(i)*36 + (j)][1])) > .2:
      print(f"{chr(i+65)}{(j+1):02d} / {abs(float(tempArr[(i)*18 + (j)][1]))}")

plt.scatter(xList, yList, s = .2)
plt.show()

# Relevant part


# Irrelevant part again